# Enriching The Graph: Add Image Description

Sometimes, the information in our Knowledge Base is presented in the format of graphic. For example, troubleshooting steps are typically depicted as a workflow image. It can be hard to search this information if the information can only retrieved visually. We can leverage LLM to automatically generate description of the images so that it can be easily queried later.

In [ ]:
%pip install openai neo4j

In [1]:
from neo4j import GraphDatabase
from openai import OpenAI

In [ ]:
username = 'neo4j'
password = ''
uri = ''
openai_api_key = ''

Just for the sake of demonstration, I'll update the URL of the image to point to public image hosting so that it can be accessed by OpenAI API. The other images used in this demo are Microsoft's stock images that can be accessed publicly.

The following image depicts a troubleshooting process for IoT Manufactr product.

In [ ]:
img_url='https://i.imgur.com/dVpGWgf.png'

Update the image URL of the respective web part:

In [9]:
with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    driver.execute_query(
        '''
        MATCH (w:WebPage {id: '91ce1a95-8b21-4c3c-bb3a-69c0cd294f98'})
        MATCH (wp:WebPart)-[:PARTS_OF]->(w) WHERE wp.order=10
        MATCH (wp)-[:LINKED_TO]->(i:Image)
        SET wp.text='Image source: ' + $img_url
        SET i.url=$img_url
        
        ''',img_url=img_url
    )


Next, get all image URLs from the graph.

In [24]:
images = None

with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    images = driver.execute_query(
        '''MATCH (i:Image) RETURN i.url AS url
        '''
    )

In [25]:
image_links = []
for image in images.records:
    image_links.append(image['url'])

In [26]:
image_links

['https://cdn.hubblecontent.osi.office.net/m365content/publish/76bca189-84bb-44a3-935c-c54e7864cc9f/1273582733.jpg',
 'https://i.imgur.com/dVpGWgf.png',
 'https://cdn.hubblecontent.osi.office.net/m365content/publish/984eb06b-e13d-4c1c-a05c-4f1f490993f1/1025633076.jpg',
 'https://cdn.hubblecontent.osi.office.net/m365content/publish/57db0400-6f15-406f-9050-559272b52e4c/1217424559.jpg',
 'https://cdn.hubblecontent.osi.office.net/m365content/publish/74f4e0ab-93a2-4d44-94d5-fda2981119c7/694075824.jpg']

Next, we'll use OpenAI to describe the content of the images above.

In [27]:
client = OpenAI(api_key=openai_api_key)

def get_image_description(img_url):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {
                'role': 'user',
                'content': [
                    {
                        'type': 'text',
                        'text': 'What is in this image?'
                    },
                    {
                        'type': 'image_url',
                        'image_url': {
                            'url': img_url
                        }
                    }
                ]
            }
        ],
        max_tokens=500,
    )

    return response.choices[0]

In [28]:
res = []

for url in image_links:
    desc = get_image_description(url)
    res.append({'url':url, 'description': desc.message.content})

In [29]:
for item in res:
    print(f'\n==== {item['url']} ====')
    print(item['description'])


==== https://cdn.hubblecontent.osi.office.net/m365content/publish/76bca189-84bb-44a3-935c-c54e7864cc9f/1273582733.jpg ====
The image appears to depict an industrial robotic arm or manipulator, likely used in manufacturing or automation settings. The arm is shown gripping a component, and there are overlay graphics that suggest data visualization, possibly representing operational metrics or programming code. The background seems to feature a high-tech environment, often found in factories or research facilities.

==== https://i.imgur.com/dVpGWgf.png ====
The image is a flowchart titled "Troubleshooting Guide for IoT Manufacturer." It outlines a series of steps to diagnose and resolve connectivity issues with an IoT device. 

Key steps include:

1. **Check connections**: Verify that cables and adapters are connected correctly to the router.
2. **LED indicators**: Assess if the LED indicators on the device are functioning properly.
3. **Internet connection**: Determine if the IoT module

Finally, append the description into the respective nodes in the graph

In [30]:
with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    total = driver.execute_query(
        '''
        UNWIND $res AS res
        MATCH (i:Image) WHERE i.url=res.url
        MATCH (i)<-[:LINKED_TO]-(wp:WebPart)
        SET wp.text=res.description
        RETURN COUNT(wp)
        ''',res=res
    )

    print(total.records)

[<Record COUNT(wp)=5>]
